In [1]:
from sqlalchemy import create_engine
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# Configure settings for RDS
jdbc_url="jdbc:postgresql://34.67.52.115/team5k"
config = {'user': 'postgres', 
          "password": "team5kteam5k", 
          "driver":"org.postgresql.Driver",
          "location": "34.67.52.115",
          "db": "team5k",
          "port": "5432"}

In [3]:
#postgres://[user]:[password]@[location]:[port]/[database]
create_engine_str = 'postgresql://' + config["user"] + ":" + config["password"] + "@" + config["location"] + ":" + config["port"] + "/" + config["db"]

In [4]:
engine = create_engine(create_engine_str)

In [5]:
print (engine.table_names())

['postal_codes', 'fec_donor_wi', 'education', 'committee_summary_2020', 'individual_contributions', 'health_metrics', 'fec_committee', 'fec_donor_mi', 'fec_donor_az', 'birth_death_rate', 'fec_donor_pa', 'fec_donor_fl', 'fec_donor_nc', 'pres_votes_6t']


In [6]:
def get_votes_intervals(votes_df, state_po):
    #.groupby("county")
    votes_states_df = votes_df[votes_df['state_po']==state_po]
    starting_yr = 2000
    ending_yr = 2020
    interval = 4
    i = starting_yr
    
    four_yr_dfs = []
    while (i <= ending_yr):
        votes_states_interval_df = votes_states_df[votes_states_df['year']==i]
        four_yr_dfs.append(votes_states_interval_df)
        i += interval
    
    return four_yr_dfs

In [7]:
fl_df = pd.read_sql_query('select * from "fec_donor_fl"',con=engine).groupby("zip")
fl_df.head()

,id,cmt_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,last_name,first_name,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub id
0,513914,c00008201,n,my,None,99034714312,15,None,montgomery,robert e,...,None,None,6/18/1999,300,None,None,None,None,None,3061920110007110000
1,513915,c00310490,n,my,None,99020070026,15,None,egan,john m,...,antec,None,3/20/1999,1000,None,None,None,None,None,3061920110007060000
2,513916,c00310490,n,my,None,99020070026,15,None,egan,john m,...,antec,None,3/20/1999,1000,None,None,None,None,None,3061920110007060000
3,513917,c00310490,n,my,None,99020070055,15,None,mishkin,david g,...,northwest airlines,None,3/29/1999,500,None,None,None,None,None,3061920110007090000
4,513918,c00310490,n,my,None,99020070005,15,None,arison,micky,...,carnival corporation,None,6/22/1999,1000,None,None,None,None,None,3061920110007090000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,514009,c00266940,a,ye,None,20035143377,15,None,campbell,r l,...,campbell roofing,None,8/23/1999,1000,None,None,None,None,None,3061920110006760000
96,514010,c00266940,a,ye,None,20035143384,15,None,laimbeer,william,...,colson bus sys,None,11/23/1999,1000,None,None,None,None,None,3061920110006760000
97,514011,c00286054,a,my,None,99020092443,15,None,logan,john f,...,None,None,6/25/1999,250,None,None,None,None,None,3061920110007070000
98,514012,c00286054,a,my,None,99020092431,15,None,paulucci,gina j,...,paulucci international,None,6/4/1999,2000,None,None,None,None,None,3061920110007110000


In [8]:
#wi_df = pd.read_sql_query('select * from "fec_donor_wi"',con=engine)
#az_df = pd.read_sql_query('select * from "fec_donor_az"',con=engine)
#nc_df = pd.read_sql_query('select * from "fec_donor_nc"',con=engine)
#pa_df = pd.read_sql_query('select * from "fec_donor_pa"',con=engine)
#mi_df = pd.read_sql_query('select * from "fec_donor_mi"',con=engine)

In [9]:
health_df = pd.read_sql_query('select * from "health_metrics"',con=engine)
health_df.head()

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
0,1000,Alabama,None,81791.0,9942.794666,9840.535949,10045.053384,NaN,NaN,NaN,...,4.443264,3197324,65.413428,48517,1.061048,1.006759,1.115337,51.633032,1957932.0,40.963183
1,42067,Pennsylvania,Juniata,301.0,6943.173764,5646.653331,8239.694197,1.0,NaN,NaN,...,4.003400,23202,93.920013,382,1.649253,0.998613,2.299893,49.866418,20264.0,82.253613
2,42069,Pennsylvania,Lackawanna,3282.0,8878.231356,8401.648792,9354.813920,4.0,NaN,NaN,...,8.102736,177750,84.324432,4543,2.268653,1.929494,2.607811,51.454745,34965.0,16.305488
3,46021,South Dakota,Campbell,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,2.033406,1316,95.570080,0,0.000000,0.000000,3.124080,48.002905,1466.0,100.000000
4,42071,Pennsylvania,Lancaster,5459.0,6128.340367,5882.890633,6373.790101,1.0,NaN,NaN,...,10.810458,443304,81.556120,11626,2.310739,2.099442,2.522036,51.004954,110419.0,21.257111


In [10]:
committee_df = pd.read_sql_query('select * from "fec_committee"',con=engine)
committee_df.head()

,cmte_id,cmte_nm,cmte_tp,cmte_city,cmte_st,cmte_zip,fec_election_yr,ttl_contb,indv_contb
0,C00032607,SYNOVUS FINANCIAL CORP. FUND FOR EFFECTIVE LEA...,Q,COLUMBUS,GA,31902,2020,49295.08,49295.08
1,C00032979,D.R.I.V.E. - DEMOCRAT REPUBLICAN INDEPENDENT V...,Q,WASHINGTON,DC,20001,2020,11466792.68,11466792.68
2,C00659813,CATHERINE KRANTZ FOR CONGRESS,H,EMORY,TX,75440,2020,0.00,0.00
3,C00647180,FLORIDA FIRST FLORIDA FOREVER,O,TAMPA,FL,33606,2020,0.00,0.00
4,C00624627,PALMER VICTORY COMMITTEE,N,HOMEWOOD,AL,35209,2020,270300.00,270300.00


In [11]:
votes_df = pd.read_sql_query('select * from "pres_votes_6t"',con=engine)
votes_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache,4001,President,Al Gore,democrat,13025,19456,20191203
1,2000,Arizona,AZ,Apache,4001,President,George W. Bush,republican,5947,19456,20191203
2,2000,Arizona,AZ,Apache,4001,President,Ralph Nader,green,245,19456,20191203
3,2000,Arizona,AZ,Apache,4001,President,Other,NA,239,19456,20191203
4,2000,Arizona,AZ,Cochise,4003,President,Al Gore,democrat,13360,33241,20191203


In [12]:
#TODO ensure the AZ vs az
votes_az_df = get_votes_intervals(votes_df, "AZ")
votes_az_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Arizona,AZ,Apache,4001,President,Al Gore,democrat,13025,19456,20191203
1,2000,Arizona,AZ,Apache,4001,President,George W. Bush,republican,5947,19456,20191203
2,2000,Arizona,AZ,Apache,4001,President,Ralph Nader,green,245,19456,20191203
3,2000,Arizona,AZ,Apache,4001,President,Other,NA,239,19456,20191203
4,2000,Arizona,AZ,Cochise,4003,President,Al Gore,democrat,13360,33241,20191203


In [13]:
votes_az_df[1].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
1616,2004,Arizona,AZ,Apache,4001,President,John Kerry,democrat,15658,24198,20191203
1617,2004,Arizona,AZ,Apache,4001,President,George W. Bush,republican,8384,24198,20191203
1618,2004,Arizona,AZ,Apache,4001,President,Other,NA,156,24198,20191203
1619,2004,Arizona,AZ,Cochise,4003,President,John Kerry,democrat,17514,44483,20191203
1620,2004,Arizona,AZ,Cochise,4003,President,George W. Bush,republican,26556,44483,20191203


In [14]:
votes_az_df[2].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
2828,2008,Arizona,AZ,Apache,4001,President,Barack Obama,democrat,15390,24262,20191203
2829,2008,Arizona,AZ,Apache,4001,President,John McCain,republican,8551,24262,20191203
2830,2008,Arizona,AZ,Apache,4001,President,Other,NA,321,24262,20191203
2831,2008,Arizona,AZ,Cochise,4003,President,Barack Obama,democrat,18943,48820,20191203
2832,2008,Arizona,AZ,Cochise,4003,President,John McCain,republican,29026,48820,20191203


In [15]:
votes_az_df[3].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
4040,2012,Arizona,AZ,Apache,4001,President,Barack Obama,democrat,17147,25848,20191203
4041,2012,Arizona,AZ,Apache,4001,President,Mitt Romney,republican,8250,25848,20191203
4042,2012,Arizona,AZ,Apache,4001,President,Other,NA,451,25848,20191203
4043,2012,Arizona,AZ,Cochise,4003,President,Barack Obama,democrat,18546,49003,20191203
4044,2012,Arizona,AZ,Cochise,4003,President,Mitt Romney,republican,29497,49003,20191203


In [16]:
votes_az_df[4].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
5252,2016,Arizona,AZ,Apache,4001,President,Hillary Clinton,democrat,17083,27661,20191203
5253,2016,Arizona,AZ,Apache,4001,President,Donald Trump,republican,8240,27661,20191203
5254,2016,Arizona,AZ,Apache,4001,President,Other,NA,2338,27661,20191203
5255,2016,Arizona,AZ,Cochise,4003,President,Hillary Clinton,democrat,17450,50015,20191203
5256,2016,Arizona,AZ,Cochise,4003,President,Donald Trump,republican,28092,50015,20191203


In [17]:
votes_fl_df = get_votes_intervals(votes_df, "FL")
votes_fl_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
60,2000,Florida,FL,Alachua,12001,President,Al Gore,democrat,47365,85729,20191203
61,2000,Florida,FL,Alachua,12001,President,George W. Bush,republican,34124,85729,20191203
62,2000,Florida,FL,Alachua,12001,President,Ralph Nader,green,3226,85729,20191203
63,2000,Florida,FL,Alachua,12001,President,Other,NA,1014,85729,20191203
64,2000,Florida,FL,Baker,12003,President,Al Gore,democrat,2392,8154,20191203


In [18]:
votes_nc_df = get_votes_intervals(votes_df, "MI")
votes_nc_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
328,2000,Michigan,MI,Alcona,26001,President,Al Gore,democrat,2696,5997,20191203
329,2000,Michigan,MI,Alcona,26001,President,George W. Bush,republican,3152,5997,20191203
330,2000,Michigan,MI,Alcona,26001,President,Ralph Nader,green,124,5997,20191203
331,2000,Michigan,MI,Alcona,26001,President,Other,NA,25,5997,20191203
332,2000,Michigan,MI,Alger,26003,President,Al Gore,democrat,2071,4366,20191203


In [19]:
votes_nc_df = get_votes_intervals(votes_df, "NC")
votes_nc_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
660,2000,North Carolina,NC,Alamance,37001,President,Al Gore,democrat,17459,47091,20191203
661,2000,North Carolina,NC,Alamance,37001,President,George W. Bush,republican,29305,47091,20191203
662,2000,North Carolina,NC,Alamance,37001,President,Ralph Nader,green,NA,47091,20191203
663,2000,North Carolina,NC,Alamance,37001,President,Other,NA,327,47091,20191203
664,2000,North Carolina,NC,Alexander,37003,President,Al Gore,democrat,4166,13492,20191203


In [20]:
votes_pa_df = get_votes_intervals(votes_df, "PA")
votes_pa_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
1060,2000,Pennsylvania,PA,Adams,42001,President,Al Gore,democrat,11682,33433,20191203
1061,2000,Pennsylvania,PA,Adams,42001,President,George W. Bush,republican,20848,33433,20191203
1062,2000,Pennsylvania,PA,Adams,42001,President,Ralph Nader,green,696,33433,20191203
1063,2000,Pennsylvania,PA,Adams,42001,President,Other,NA,207,33433,20191203
1064,2000,Pennsylvania,PA,Allegheny,42003,President,Al Gore,democrat,329963,582478,20191203


In [21]:
votes_mi_df = get_votes_intervals(votes_df, "MI")
votes_mi_df[0].head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
328,2000,Michigan,MI,Alcona,26001,President,Al Gore,democrat,2696,5997,20191203
329,2000,Michigan,MI,Alcona,26001,President,George W. Bush,republican,3152,5997,20191203
330,2000,Michigan,MI,Alcona,26001,President,Ralph Nader,green,124,5997,20191203
331,2000,Michigan,MI,Alcona,26001,President,Other,NA,25,5997,20191203
332,2000,Michigan,MI,Alger,26003,President,Al Gore,democrat,2071,4366,20191203


In [22]:
zips_df = pd.read_sql_query('select * from "postal_codes"',con=engine)
zips_df.head()

,fips,state,county,num_deaths,years_of_potential_life_lost_rate,95percent_ci_low,95percent_ci_high,quartile,ypll_rate_aian,ypll_rate_aian_95percent_ci_low,...,percent_hispanic,num_non_hispanic_white,percent_non_hispanic_white,num_not_proficient_in_english,percent_not_proficient_in_english,95percent_ci_low_39,95percent_ci_high_39,percent_female,num_rural,percent_rural
0,42067,Pennsylvania,Juniata,301.0,6943.173764,5646.653331,8239.694197,1.0,NaN,NaN,...,4.003400,23202,93.920013,382,1.649253,0.998613,2.299893,49.866418,20264.0,82.253613
1,42069,Pennsylvania,Lackawanna,3282.0,8878.231356,8401.648792,9354.813920,4.0,NaN,NaN,...,8.102736,177750,84.324432,4543,2.268653,1.929494,2.607811,51.454745,34965.0,16.305488
2,42075,Pennsylvania,Lebanon,1716.0,7039.360299,6509.617529,7569.103069,2.0,NaN,NaN,...,13.782074,115105,81.453359,2855,2.191955,1.811654,2.572257,50.854834,35470.0,26.555762
3,42071,Pennsylvania,Lancaster,5459.0,6128.340367,5882.890633,6373.790101,1.0,NaN,NaN,...,10.810458,443304,81.556120,11626,2.310739,2.099442,2.522036,51.004954,110419.0,21.257111
4,42073,Pennsylvania,Lawrence,1439.0,10144.125906,9305.867051,10982.384761,4.0,NaN,NaN,...,1.526966,79146,91.833751,392,0.472665,0.242681,0.702649,51.601225,36733.0,40.318084


In [ ]:
#Define the Linear Regression Structured MAchine Learning

In [ ]:
model = LinearRegression()

In [ ]:
#Define X, y

In [ ]:
model.fit(X, y)

In [ ]:
y_pred = model.predict(X)
print(y_pred.shape)

In [ ]:
plt.scatter(X, y)
plt.plot(X, y_pred, color='red')
plt.show()

In [ ]:
print(model.coef_)
print(model.intercept_)

In [23]:
#Neural Networking Code
# Generate our categorical variable list
votes_mi_cat = votes_mi_df.dtypes[votes_mi_df.dtypes == "object"].index.tolist()

AttributeError: 'list' object has no attribute 'dtypes'

In [ ]:
# Check the number of unique values in each column
votes_mi_df[votes_mi_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(votes_mi_df[votes_mi_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(votes_mi_cat)
encode_df.head()

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
#TODO would we be doing a classification to determine what party?
y = votes_mi_df["party"].values
X = votes_mi_df.drop(["party"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
merged_df = pd.merge(df1, df2, 
                     left_on = 'names', 
                     right_on = 'name', 
                     how='left')